<center>

# Web Intelligence

# Who will win the Tennis Australian Open 2020?

## 00 Dataset Download and Examination


### Riccardo Spolaor (864877)

</center>

## Index

1. [Scaricare il dataset e rendere i dati facilmente accessibili](#s_1)
1. Section 2

<a id="s_1"></a>

## Scaricare il dataset e rendere i dati facilmente accessibili

Il primo step consiste nello scaricare i vari dataset, importarli in oggetti DataFrame della libreria pandas ed unirli in un solo DataFrame.

Tengo traccia dei differenti dataset scaricati assegnando ad ognuno una feature ***csvID***.

Ignoro i dati dell'anno 2000 in quanto sono privi di dati riguardo alle scommesse.

In [1]:
import urllib.request, zipfile, pandas

def get_file(url):
    filehandle, _ = urllib.request.urlretrieve(url)
    zip_file_obj = zipfile.ZipFile(filehandle, 'r')
    file = zip_file_obj.open(zip_file_obj.namelist()[0])
    return file

dataframe = pandas.DataFrame()
dataframe.insert(0, column ='csvID', value = 0)

for date in range(2001,2019+1):
    file = get_file('http://tennis-data.co.uk/' + str(date) + '/' + str(date) + '.zip')
    dataframe = dataframe.append(pandas.read_excel(file).assign(csvID = date-2001), sort = False, ignore_index = True)


dataframe.head()

,csvID,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,...,UBW,UBL,LBW,LBL,SJW,SJL,MaxW,MaxL,AvgW,AvgL
0,0,1.0,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,1.0,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,1.0,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,1.0,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,1.0,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
dataframe.columns

Index(['csvID', 'ATP', 'Location', 'Tournament', 'Date', 'Series', 'Court',
       'Surface', 'Round', 'Best of', 'Winner', 'Loser', 'WRank', 'LRank',
       'W1', 'L1', 'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5', 'L5', 'Wsets',
       'Lsets', 'Comment', 'CBW', 'CBL', 'GBW', 'GBL', 'IWW', 'IWL', 'SBW',
       'SBL', 'B365W', 'B365L', 'B&WW', 'B&WL', 'EXW', 'EXL', 'PSW', 'PSL',
       'WPts', 'LPts', 'UBW', 'UBL', 'LBW', 'LBL', 'SJW', 'SJL', 'MaxW',
       'MaxL', 'AvgW', 'AvgL'],
      dtype='object')

### Correzione dei tipi assegnati in modo scorretto
Controllo se alcune feature hanno assegnati tipi differenti da quelli attesi e correggo eventuali valori errati

In [3]:
print('Feature di tipo Object:\n', dataframe.dtypes[dataframe.dtypes == object].index.tolist())

dataframe.head()[dataframe.dtypes[dataframe.dtypes == object].index]

Feature di tipo Object:
 ['Location', 'Tournament', 'Series', 'Court', 'Surface', 'Round', 'Winner', 'Loser', 'LRank', 'W2', 'L2', 'W3', 'L3', 'Lsets', 'Comment', 'EXW']


,Location,Tournament,Series,Court,Surface,Round,Winner,Loser,LRank,W2,L2,W3,L3,Lsets,Comment,EXW
0,Adelaide,AAPT Championships,International,Outdoor,Hard,1st Round,Clement A.,Gaudenzi A.,101,6,0,6,3,1,Completed,NaN
1,Adelaide,AAPT Championships,International,Outdoor,Hard,1st Round,Goldstein P.,Jones A.,442,6,2,NaN,NaN,0,Completed,NaN
2,Adelaide,AAPT Championships,International,Outdoor,Hard,1st Round,Haas T.,Smith L.,485,6,3,NaN,NaN,0,Completed,NaN
3,Adelaide,AAPT Championships,International,Outdoor,Hard,1st Round,Henman T.,Rusedski G.,69,7,6,NaN,NaN,0,Completed,NaN
4,Adelaide,AAPT Championships,International,Outdoor,Hard,1st Round,Hewitt L.,Arthurs W.,83,6,3,NaN,NaN,0,Completed,NaN


***LRank***, ***W2***, ***L2***, ***W3***, ***L3***, ***Lsets*** e ***EXW*** presentano valori numerici, ma sono categorizzati come *Object*.

Controllo quali sono i valori non numerici che non permettono il corretto casting dei tipi e procedo alla loro correzione

In [4]:
import numbers
import numpy as np

def checkValues(attribute_names):
    for a in attribute_names:
        print('"'+a+'": ', 'Wrong Values -> ' , set([x for x in dataframe[a] if not isinstance(x, numbers.Number)]))
    
checkValues(['LRank', 'W2', 'L2', 'W3', 'L3', 'Lsets', 'EXW'])

"LRank":  Wrong Values ->  {'NR'}
"W2":  Wrong Values ->  {' '}
"L2":  Wrong Values ->  {' '}
"W3":  Wrong Values ->  {' '}
"L3":  Wrong Values ->  {' '}
"Lsets":  Wrong Values ->  {'`1'}
"EXW":  Wrong Values ->  {'2.,3'}


In [5]:
import re

def reassignNans(attribute_names):
    for a in attribute_names:
        wrong_values = set([x for x in dataframe[a] if not isinstance(x, numbers.Number)])
        dataframe.loc[dataframe[a].isin(wrong_values), a] = np.nan
        dataframe[a] = dataframe[a].astype(float)

def correctTypos(attribute_names):
    for a in attribute_names:
        wrong_values = set([x for x in dataframe[a] if not isinstance(x, numbers.Number)])
        indexes = dataframe[dataframe[a].isin(wrong_values)].index.values
        dataframe.iloc[indexes]
        for i in indexes:
            dataframe.iloc[i,dataframe.columns.get_loc(a)] = float(re.sub("[^0-9^.]", "", str(dataframe.iloc[i][a])))            
        dataframe[a] = dataframe[a].astype(float)

In [6]:
print('WRank, LRank, W2, L2, W3 and L3 have strings to represent NaN values.')
print('It is possible to set these strings to NaN and assign "float" as the features type :\n')

reassignNans(['WRank', 'LRank', 'W2', 'L2', 'W3', 'L3'])
checkValues(['WRank', 'LRank', 'W2', 'L2', 'W3', 'L3'])

WRank, LRank, W2, L2, W3 and L3 have strings to represent NaN values.
It is possible to set these strings to NaN and assign "float" as the features type :

"WRank":  Wrong Values ->  set()
"LRank":  Wrong Values ->  set()
"W2":  Wrong Values ->  set()
"L2":  Wrong Values ->  set()
"W3":  Wrong Values ->  set()
"L3":  Wrong Values ->  set()


In [7]:
print('EXW and Lsets have typos.')
print('It is possible to correct these typos and assign "float" as the features type :\n')

correctTypos(['EXW', 'Lsets'])
checkValues(['EXW', 'Lsets'])

EXW and Lsets have typos.
It is possible to correct these typos and assign "float" as the features type :

"EXW":  Wrong Values ->  set()
"Lsets":  Wrong Values ->  set()


In [8]:
%store dataframe

Stored 'dataframe' (DataFrame)


In [9]:
%store

Stored variables and their in-db values:
dataframe             ->        csvID   ATP  Location           Tournament 
